# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Import dependencies
import hvplot.pandas
import pandas as pd
import requests


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vadso,70.0744,29.7487,1.09,80,40,6.69,NO,2023-11-01 09:44:20
1,1,blackmans bay,-43.0167,147.3167,13.07,66,1,0.45,AU,2023-11-01 09:44:20
2,2,susuman,62.7833,148.1667,-19.38,99,79,1.09,RU,2023-11-01 09:44:20
3,3,aqaba,29.5267,35.0078,30.92,33,0,4.12,JO,2023-11-01 09:40:19
4,4,port-aux-francais,-49.3500,70.2167,4.02,93,100,7.52,TF,2023-11-01 09:44:21


---

### Step 1: Create a map that displays a point for every city in the `city_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    alpha = 0.6,
    frame_width = 700,
    frame_height = 600,
    color = "City"
     
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
city_narrow = city_df[(city_df["Max Temp"] < 27) &
                             (city_df["Max Temp"] > 21) &
                             (city_df["Wind Speed"] < 4.5) &
                             (city_df["Cloudiness"] == 0) ]

# Drop any rows with null values
city_narrow.dropna()

# Display sample data
city_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,la'ie,21.6477,-157.9253,25.98,84,0,1.32,US,2023-11-01 09:44:22
57,57,jiaozuo,35.2397,113.2331,25.20,38,0,2.16,CN,2023-11-01 09:44:38
84,84,wailua homesteads,22.0669,-159.3780,22.24,82,0,4.12,US,2023-11-01 09:44:46
154,154,jimo,36.3897,120.4622,22.88,60,0,3.00,CN,2023-11-01 09:45:01
164,164,shaoxing,30.0110,120.5715,23.97,47,0,1.74,CN,2023-11-01 09:45:05
203,203,holualoa,19.6228,-155.9522,23.91,81,0,3.09,US,2023-11-01 09:45:20
208,208,calvinia,-31.4707,19.7760,25.90,21,0,3.60,ZA,2023-11-01 09:45:21
274,274,lihue,21.9789,-159.3672,22.58,82,0,4.12,US,2023-11-01 09:45:42
478,478,cabo san lucas,22.8909,-109.9124,25.84,81,0,1.66,MX,2023-11-01 09:43:29
496,496,ribeira brava,32.6500,-17.0667,22.62,79,0,0.65,PT,2023-11-01 09:46:51


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_narrow [["City","Country","Lat","Lng","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,la'ie,US,21.6477,-157.9253,84,
57,jiaozuo,CN,35.2397,113.2331,38,
84,wailua homesteads,US,22.0669,-159.3780,82,
154,jimo,CN,36.3897,120.4622,60,
164,shaoxing,CN,30.0110,120.5715,47,
203,holualoa,US,19.6228,-155.9522,81,
208,calvinia,ZA,-31.4707,19.7760,21,
274,lihue,US,21.9789,-159.3672,82,
478,cabo san lucas,MX,22.8909,-109.9124,81,
496,ribeira brava,PT,32.6500,-17.0667,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "api_key":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la'ie - nearest hotel: No hotel found
jiaozuo - nearest hotel: Days Hotel & Suites Jiaozuo
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
jimo - nearest hotel: DoubleTree by Hilton Qingdao
shaoxing - nearest hotel: 绍兴国际大酒店
holualoa - nearest hotel: Kona Hotel
calvinia - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
cabo san lucas - nearest hotel: Comfort Rooms
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
7,la'ie,US,21.6477,-157.9253,84,No hotel found
57,jiaozuo,CN,35.2397,113.2331,38,Days Hotel & Suites Jiaozuo
84,wailua homesteads,US,22.0669,-159.3780,82,Hilton Garden Inn Kauai Wailua Bay
154,jimo,CN,36.3897,120.4622,60,DoubleTree by Hilton Qingdao
164,shaoxing,CN,30.0110,120.5715,47,绍兴国际大酒店
203,holualoa,US,19.6228,-155.9522,81,Kona Hotel
208,calvinia,ZA,-31.4707,19.7760,21,No hotel found
274,lihue,US,21.9789,-159.3672,82,Kauai Palms
478,cabo san lucas,MX,22.8909,-109.9124,81,Comfort Rooms
496,ribeira brava,PT,32.6500,-17.0667,79,Cheerfulway Bravamar Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   frame_width = 700,
                                   frame_height = 600,
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   color = "City",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )


# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)